In [178]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [179]:
import re
import os
import string
import pandas as pd
import numpy as np
import syllables

#Extract Data from our Drive

In [180]:
extracted_texts_folder = '/content/drive/MyDrive/Blackcoffer/Data'

extracted_texts = {}
for file_name in os.listdir(extracted_texts_folder):
    file_path = os.path.join(extracted_texts_folder, file_name)

    # Check if the item in the folder is a file
    if os.path.isfile(file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
            extracted_texts[file_name] = file.read()


#Text Cleanning

In [181]:
def remove_html_tags(text):
    pattern = re.compile('<.*?>')
    return pattern.sub(r'', text)


def remove_url(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'', text)


def remove_double_newlines(text):
    return text.replace('\n\n', ' ').replace('\n', ' ')

exclude = set(string.punctuation)
def remove_punctuation(text):
    for char in exclude:
        text = text.replace(char, '')
    return text


import unicodedata
# Assuming 'text' is your input text
def remove_unicodedata(text):
  return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')


In [182]:
def preprocess_text(input_text):
    # Remove HTML tags
    dict_html = remove_html_tags(input_text)

    # Remove URLs
    dict_url = remove_url(dict_html)

    # Remove punctuation
    dict_pun = remove_punctuation(dict_url)

    # Remove double newlines
    dict_newline = remove_double_newlines(dict_pun)

    dict_unicode = remove_unicodedata(dict_newline)

    return { dict_unicode

    }

preprocessed_texts = {key: str(preprocess_text(text)).lower()  for key, text in extracted_texts.items()}


#Cleaning using Stop Words Lists
The Stop Words Lists (found in the folder StopWords) are used to clean the text so that Sentiment Analysis can be performed by excluding the words found in Stop Words List.


In [183]:
stopword = '/content/drive/MyDrive/Blackcoffer/StopWords'

# List to store merged stopwords
merged_stopwords = []

# Merge and make all letters small
for file_name in os.listdir(stopword):
    with open(os.path.join(stopword, file_name), 'r', encoding='latin-1') as file:
        stopwords = file.read().splitlines()
        merged_stopwords.extend(stopwords)

# Remove duplicates and convert to lowercase
merged_stopwords = list(set(map(str.lower, merged_stopwords)))

In [184]:
cleaned_stopwords = [remove_url(word) for word in merged_stopwords]


In [185]:
def clean_text(text, cleaned_stopwords):
    cleaned_text_no_stopword = ' '.join([word for word in text.split() if word.lower() not in cleaned_stopwords])
    return cleaned_text_no_stopword
dict_stopword_cleaned = {key: clean_text(text, cleaned_stopwords) for key, text in preprocessed_texts.items()}


#Creating a dictionary of Positive and Negative words
The Master Dictionary (found in the folder MasterDictionary) is used for creating a dictionary of Positive and Negative words. We add only those words in the dictionary if they are not found in the Stop Words Lists.


In [186]:
positive_words_path = '/content/drive/MyDrive/Blackcoffer/MasterDictionary/positive-words.txt'

positive_words = set()
with open(positive_words_path, 'r', encoding='utf-8') as file:
    positive_words.update(file.read().splitlines())

In [187]:
negative_words_path = '/content/drive/MyDrive/Blackcoffer/MasterDictionary/negative-words.txt'

negative_words = set()
with open(negative_words_path, 'r', encoding= 'latin-1') as file:
    negative_words.update(file.read().splitlines())

In [188]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [189]:
# Create a directory to save tokenized text files
output_directory = '/content/drive/MyDrive/Blackcoffer/TokenizedText/'
os.makedirs(output_directory, exist_ok=True)

dict_tokenized = {}
# Tokenize and save each value in the dictionary
for key, text in dict_stopword_cleaned.items():
    # Tokenize the text
    tokens_file = word_tokenize(text.lower())  # Convert to lowercase for case-insensitivity

    # Save tokenized text in the dictionary
    dict_tokenized[key] = tokens_file

    # Save the tokens to a file (optional)
    output_file_path = os.path.join(output_directory, f'{key}_tokens_file.txt')
    with open(output_file_path, 'w', encoding='utf-8') as output_file:
        output_file.write('\n'.join(tokens_file))



#Extracting Derived variables

Positive Score: This score is calculated by assigning the value of +1 for each word if found in the Positive Dictionary and then adding up all the values.


Negative Score: This score is calculated by assigning the value of -1 for each word if found in the Negative Dictionary and then adding up all the values. We multiply the score with -1 so that the score is a positive number.



Polarity Score = (Positive Score – Negative Score)/ ((Positive Score + Negative Score) + 0.000001)        Range is from -1 to +1

Subjectivity Score: Subjectivity Score = (Positive Score + Negative Score)/ ((Total Words after cleaning) + 0.000001)
Range is from 0 to +1


In [190]:
positive_scores_dict = {}
negative_scores_dict = {}
polarity_scores_dict = {}
subjectivity_scores_dict = {}

# Calculate Positive Score and Negative Score for each text
for key, tokens in dict_tokenized.items():
    # Positive Score: +1 for each positive word
    positive_score = sum(1 for word in tokens if word in positive_words)

    # Negative Score: -1 for each negative word (multiplied by -1 for positive result)
    negative_score =  sum(1 for word in tokens if word in negative_words)

    # Polarity Score: (Positive Score - Negative Score) / ((Positive Score + Negative Score) + 0.000001)
    polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)


    # Subjectivity Score: (Positive Score + Negative Score) / (Total Words after cleaning + 0.000001)
    subjectivity_score = (positive_score + negative_score) / (len(tokens) + 0.000001)


    # Store scores in dictionaries
    positive_scores_dict[key] = positive_score
    negative_scores_dict[key] = negative_score
    polarity_scores_dict[key] = polarity_score
    subjectivity_scores_dict[key] = subjectivity_score

# Analysis of Readability

### Average Sentence Length = the number of words / the number of sentences

### Percentage of Complex words = the number of complex words / the number of words

### Fog Index = 0.4 * (Average Sentence Length + Percentage of Complex words)

I am not removing Stopwords Here.

In [191]:

from nltk.tokenize import word_tokenize, sent_tokenize

def calculate_readability_metrics(text):
    sentences = sent_tokenize(text)
    total_words = len(word_tokenize(text))
    total_sentences = len(sentences)

    average_sentence_length = total_words / total_sentences

    complex_words = [word for word in word_tokenize(text) if len(word) > 2]
    percentage_complex_words = len(complex_words) / total_words

    fog_index = 0.4 * (average_sentence_length + percentage_complex_words)

    return {
        'Average Sentence Length': round(average_sentence_length),
        'Percentage of Complex Words': round(percentage_complex_words,3),
        'Fog Index': round(fog_index,3)
    }



In [192]:
# Assuming dict_stopword_cleaned is your dictionary
average_sentence_length_dict = {}
percentage_complex_words_dict = {}
fog_index_dict = {}

for key, text in extracted_texts.items():
    readability_metrics = calculate_readability_metrics(text)
    average_sentence_length_dict[key] = readability_metrics['Average Sentence Length']
    percentage_complex_words_dict[key] = readability_metrics['Percentage of Complex Words']
    fog_index_dict[key] = readability_metrics['Fog Index']

#Average Number of Words Per Sentence
The formula for calculating is:

Average Number of Words Per Sentence = the total number of words / the total number of sentences

In [193]:

def calculate_average_words_per_sentence(text):
    sentences = sent_tokenize(text)
    total_words = len(word_tokenize(text))
    total_sentences = len(sentences)

    average_words_per_sentence = total_words / total_sentences
    return { 'Average words per sentence' :round(average_words_per_sentence)}

average_words_per_sentence = {}
for key, text in extracted_texts.items():
    Averge_word_per_sentence = calculate_average_words_per_sentence(text)
    average_words_per_sentence[key] =  Averge_word_per_sentence ['Average words per sentence']


#Complex Word Count
Complex words are words in the text that contain more than two syllables.


In [194]:


def calculate_complex_word_count(tokens):
    # Count complex words (words with more than two syllables)
    complex_words_count = sum(1 for word in tokens if syllables.estimate(word) > 2)

    return complex_words_count

# Assuming dict_tokenized is your dictionary where keys are identifiers and values are tokenized texts
complex_word_counts = {}

for key, tokens in dict_tokenized.items():
    complex_word_counts[key] = calculate_complex_word_count(tokens)




#Word Count
We count the total cleaned words present in the text by
removing the stop words (using stopwords class of nltk package).
removing any punctuations like ? ! , . from the word before counting.


In [195]:
def calculate_word_count(tokens):
    # Count the total number of cleaned words
    cleaned_word_count = len(tokens)

    return cleaned_word_count

# Assuming dict_tokenized is your dictionary where keys are identifiers and values are tokenized texts
word_counts = {}

for key, tokens in dict_tokenized.items():
    word_counts[key] = calculate_word_count(tokens)

#Syllable Count Per Word
We count the number of Syllables in each word of the text by counting the vowels present in each word. We also handle some exceptions like words ending with "es","ed" by not counting them as a syllable.


In [196]:

def calculate_syllable_count_per_word(tokens):
    # Function to count syllables in a word, handling exceptions
    def count_syllables(word):
        # Handle exceptions for words ending with "es" and "ed"
        if word.endswith(('es', 'ed')):
            return max(1, syllables.estimate(word) - 1)
        else:
            return syllables.estimate(word)

    # Count syllables for each word
    syllable_count_per_word = [count_syllables(word) for word in tokens]

    return syllable_count_per_word

# Assuming dict_tokenized is your dictionary where keys are identifiers and values are tokenized texts
average_syllable_count_per_word = {}

for key, tokens in dict_tokenized.items():
    syllable_counts = calculate_syllable_count_per_word(tokens)

    # Calculate the average syllable count per word for the current key
    average_syllable_count = sum(syllable_counts) / len(syllable_counts) if len(syllable_counts) > 0 else 0

    average_syllable_count_per_word[key] = round(average_syllable_count)




#Personal Pronouns
To calculate Personal Pronouns mentioned in the text, we use regex to find the counts of the words - “I,” “we,” “my,” “ours,” and “us”. Special care is taken so that the country name US is not included in the list.


In [197]:


def calculate_personal_pronouns_count(text):

    # Define the personal pronouns and the regex pattern
    personal_pronouns = ["I", "we", "my", "ours", "us"]
    pronouns_pattern = re.compile(r"\b(?:" + "|".join(personal_pronouns) + r")\b", flags=re.IGNORECASE)

    # Find all occurrences of personal pronouns, excluding "US" as a country name
    personal_pronouns_found = [
        word.lower()
        for word in text.split()
        if pronouns_pattern.fullmatch(word.lower()) and word.lower() != "US"
    ]

    # Count the number of occurrences
    personal_pronouns_count = len(personal_pronouns_found)

    return personal_pronouns_count

# Assuming preprocessed_texts is your dictionary where keys are identifiers and values are tokenized texts
personal_pronouns_counts = {}

for key, tokens in preprocessed_texts.items():
    personal_pronouns_counts[key] = calculate_personal_pronouns_count(tokens)



#Average Word Length
Average Word Length is calculated by the formula:
Sum of the total number of characters in each word/Total number of words


In [198]:

def calculate_average_word_length(text):
    words = text.split()  # Split the text into words
    total_characters = sum(len(word) for word in words)  # Count total characters
    total_words = len(words)  # Count total words
    return round(total_characters / total_words)  # Calculate average word length

# Assuming preprocessed_texts is your dictionary where keys are identifiers and values are tokenized texts
average_word_lengths = {}

for key, texts in dict_stopword_cleaned.items():
    average_word_lengths[key] = calculate_average_word_length(texts)


#Making FINAL excel sheet

In [199]:
one  = pd.DataFrame.from_dict(positive_scores_dict, orient='index', columns=['POSITIVE SCORE'])
one  =  one.rename_axis('URL_ID')
two = pd.DataFrame.from_dict(negative_scores_dict, orient='index', columns=['NEGATIVE SCORE'])
two =  two.rename_axis('URL_ID')
three= pd.DataFrame.from_dict(polarity_scores_dict, orient='index', columns=['POLARITY SCORE'])
three=  three.rename_axis('URL_ID')
four = pd.DataFrame.from_dict(subjectivity_scores_dict, orient='index', columns=['SUBJECTIVITY SCORE'])
four = four.rename_axis('URL_ID')
five = pd.DataFrame.from_dict(average_sentence_length_dict, orient='index', columns=['AVG SENTENCE LENGTH'])
five =  five.rename_axis('URL_ID')
six = pd.DataFrame.from_dict(percentage_complex_words_dict, orient='index', columns=['PERCENTAGE OF COMPLEX WORDS'])
six= six.rename_axis('URL_ID')
seven = pd.DataFrame.from_dict(fog_index_dict, orient='index', columns=['FOG INDEX'])
seven = seven.rename_axis('URL_ID')
eight= pd.DataFrame.from_dict(average_words_per_sentence, orient='index', columns=['AVG NUMBER OF WORDS PER SENTENCE'])
eight =  eight.rename_axis('URL_ID')
nine = pd.DataFrame.from_dict(complex_word_counts, orient='index', columns=['COMPLEX WORD COUNT'])
nine=  nine.rename_axis('URL_ID')
ten = pd.DataFrame.from_dict(word_counts, orient='index', columns=['WORD COUNT'])
ten = ten.rename_axis('URL_ID')
eleven= pd.DataFrame.from_dict(average_syllable_count_per_word, orient='index', columns=['SYLLABLE PER WORD'])
eleven=  eleven.rename_axis('URL_ID')
tweleve = pd.DataFrame.from_dict(personal_pronouns_counts, orient='index', columns=['PERSONAL PRONOUNS'])
tweleve = tweleve.rename_axis('URL_ID')
thirteen = pd.DataFrame.from_dict(average_word_lengths, orient='index', columns=['AVG WORD LENGTH'])
thirteen =thirteen.rename_axis('URL_ID')





In [200]:
# Create a list of the DataFrames to merge:
dataframes = [one, two, three, four, five, six, seven, eight, nine, ten, eleven, tweleve, thirteen]

# Merge them using pd.concat:
merged_df = pd.concat(dataframes, axis=1)

# Print the resulting DataFrame:


In [201]:
# Create empty DataFrames with desired index and NaN values:
missing_df_36 = pd.DataFrame(index=['blackassign0036.txt'], columns=merged_df.columns).fillna(np.nan)
missing_df_49 = pd.DataFrame(index=['blackassign0049.txt'], columns=merged_df.columns).fillna(np.nan)

# Insert the DataFrames at the specified positions:
merged_df = pd.concat([
    merged_df.iloc[:35],
    missing_df_36,
    merged_df.iloc[35:48],
    missing_df_49,
    merged_df.iloc[48:]
])

In [202]:
merged_df = merged_df.sort_index()
merged_df = merged_df.reset_index()
merged_df.rename(columns={'index': 'URL_ID'}, inplace=True)


In [203]:
def remove_extension(filename):
  return filename.split(".")[0]

merged_df['URL_ID'] = merged_df['URL_ID'].apply(remove_extension)


In [204]:
# Define the file path
excel_path = "/content/drive/MyDrive/Blackcoffer/Input.xlsx"

# Read the Excel file into a DataFrame
df_excel = pd.read_excel(excel_path)
df_excel

,URL_ID,URL
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...
...,...,...
95,blackassign0096,https://insights.blackcoffer.com/what-is-the-r...
96,blackassign0097,https://insights.blackcoffer.com/impact-of-cov...
97,blackassign0098,https://insights.blackcoffer.com/contribution-...
98,blackassign0099,https://insights.blackcoffer.com/how-covid-19-...


In [205]:
# Ensure identical columns for merging
common_columns = ["URL_ID"]  # Adjust if other columns need merging

final_df= pd.merge(df_excel, merged_df, on="URL_ID")

In [206]:
final_df

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,26.0,6.0,0.625000,0.064646,18.0,0.671,7.340,18.0,177.0,495.0,2.0,11.0,7.0
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,51.0,31.0,0.243902,0.117143,21.0,0.718,8.885,21.0,370.0,700.0,3.0,4.0,8.0
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,36.0,23.0,0.220339,0.096721,22.0,0.735,9.016,22.0,371.0,610.0,3.0,13.0,8.0
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,36.0,74.0,-0.345455,0.185811,24.0,0.718,9.785,24.0,342.0,592.0,3.0,5.0,8.0
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,18.0,8.0,0.384615,0.076923,20.0,0.754,8.137,20.0,167.0,338.0,2.0,6.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,blackassign0096,https://insights.blackcoffer.com/what-is-the-r...,25.0,54.0,-0.367089,0.146296,24.0,0.738,10.023,24.0,264.0,540.0,2.0,2.0,7.0
96,blackassign0097,https://insights.blackcoffer.com/impact-of-cov...,21.0,35.0,-0.250000,0.139303,32.0,0.713,12.969,32.0,157.0,402.0,2.0,5.0,7.0
97,blackassign0098,https://insights.blackcoffer.com/contribution-...,5.0,2.0,0.428571,0.033333,21.0,0.730,8.637,21.0,100.0,210.0,2.0,0.0,7.0
98,blackassign0099,https://insights.blackcoffer.com/how-covid-19-...,11.0,3.0,0.571429,0.060606,22.0,0.736,8.952,22.0,92.0,231.0,2.0,4.0,7.0


In [207]:
final_df.to_excel("Output Data Structure.xlsx", index=False)